# Imports 

Aim: Beamsplitters vs time plots

In [1]:
print('hi')

hi


In [2]:
%matplotlib inline
import os
import sys
import inspect
import numpy as np
from scipy.special import factorial
import h5py

#data_path = '/data'     ... data path specified later
#data_path
#initial_pulse = '../pulses/example_pulses/transmon_cat_initial_pulse.h5'
from h5py import File
import matplotlib.pyplot as plt
from pylab import*
from qutip import*

from scipy import interpolate
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)


In [3]:
sys.path.append(r'C:\_Lib\python\quantum-optimal-control')

In [4]:
from quantum_optimal_control.helper_functions.grape_functions import *
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control import*

In [5]:
from IPython.display import clear_output

In [6]:
%load_ext autoreload
%autoreload 2

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

In [14]:
import pandas as pd

# 3DMM2 parameters

In [8]:
# chi = Peak splitting MHz
chis = array([-33, -33]) 
kappas  = array([0,0]) # kHz

# Multimode circle grape

In [9]:
#TODO: Figure out why relative import doesn't work
sys.path.append(r'C:\_Lib\python\Multimode-Conditional-Displacements\hpc_runs\multimode_circle_grape\class_description')
from circle_grape_v4 import *

done


In [10]:
transmon_levels = 2

In [13]:
steps = int(1000/2) # not important for this notebook

mode = 2
mode_levels = 10
chi,kappa = chis[:mode],kappas[:mode]
chi = [i*1e-6 for i in chi]
kappa = [i*1e-6 for i in kappa]
circle_grape_params = {"chis":chi,"kappas":kappa,"alpha":30,"delta_c":0.01}

states_forbidden_list = []

convergence = {'rate': 0.1, 'update_step': 20, 'max_iterations': 250,
               'conv_target': 1e-3, 'learning_rate_decay': 500.0}




initial_guess = None


reg_coeffs = {'dwdt': 0.1, 'd2wdt2': 1.0e-3, 'forbid_dressed': False,
              'states_forbidden_list':states_forbidden_list,
              'forbidden_coeff_list': [1.0*steps] * len(states_forbidden_list)}

op = multimode_circle_grape_optimal_control(mode_state_num = mode_levels,
                                            number_of_modes = mode,hparams = circle_grape_params,add_disp_kerr=False)



In [15]:
#open dataset of resilts
filename = "beam_splitters"
df = pd.read_csv(filename)
df

,time,steps,alpha,detuning,qubit_drive_amp,err,filenum
0,1000,500,30,0.01,0.01,0.621863,0
1,1500,750,30,0.01,0.01,0.074013,1
2,2000,1000,30,0.01,0.01,0.169375,2
3,2500,1250,30,0.01,0.01,0.005698,3
4,1000,500,30,0.01,0.01,1.000000,4
5,1500,750,30,0.01,0.01,1.000000,5
6,2000,1000,30,0.01,0.01,1.000000,6
7,2500,1250,30,0.01,0.01,1.000000,7
8,1000,500,30,0.01,0.01,1.000000,8
9,1500,750,30,0.01,0.01,1.000000,9


In [18]:
data_filename = '00000_beamsplitter_data' + str(df['filenum'][3]) + '.h5' # highest fidelity (lowest error) for 1 photon beam splitter
start_state = [0, 1]
start_state

[0, 1]

In [20]:
t_list, out = op.qutip_mesolve_new(start_state,data_filename )

In [29]:
target = tensor(basis(2,0), basis(10, 1), basis(10, 0))
rho_t = Qobj(np.array(target*target.dag()))
rho_t

Quantum object: dims = [[200], [200]], shape = (200, 200), type = oper, isherm = True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

In [31]:
out.states[-1].overlap(rho_t)

0.9913133610999814